In [ ]:
import numpy as np
import pandas as pd
import zipfile
from zipfile import ZipFile
import glob
from openpyxl import load_workbook
pd.options.display.float_format = '{:,.2f}'.format

# PERIODO

In [ ]:
start_date = '2021-10-01'
final_date = '2021-10-29'

# PARAMETRÍA COMISIONES

In [ ]:
parametria_comisiones = pd.read_csv(r'G:\My Drive\RESPALDO-PC\AMEX\PARAMETRIA AMEX.csv',dtype={'meses_msi':object,
                                                                                              'comision_porcentaje':float})
parametria_comisiones['inicio'] = pd.to_datetime(parametria_comisiones['inicio'])
#parametria_comisiones['fin'] = pd.to_datetime(parametria_comisiones['fin'])
parametria_comisiones['comision_porcentaje_mdr'] = -0.0240
parametria_comisiones['comision_porcentaje_msi'] = -parametria_comisiones['comision_porcentaje']/100
parametria_comisiones.loc[parametria_comisiones['fuente'].str.contains('EPA'), "affiliation_type"] = 'AGGREGATOR'
parametria_comisiones.loc[parametria_comisiones['fuente'].str.contains('GRRCN'), "affiliation_type"] = 'OPTBLUE'
parametria_comisiones.loc[parametria_comisiones['meses_msi'].str.contains('99'),'meses_msi'] = '0'
parametria_comisiones.loc[parametria_comisiones['meses_msi'].str.contains('0'),'comision_porcentaje_msi'] = 0
parametria_comisiones['comision_porcentaje_final'] = (parametria_comisiones['comision_porcentaje_mdr'] +
                                                      parametria_comisiones['comision_porcentaje_msi'])
parametria_comisiones['index_comisiones'] = (parametria_comisiones['affiliation_type'].astype(str).replace(' ', '',regex=True)+'_'+
                                             parametria_comisiones['meses_msi'].astype(str).replace(' ', '',regex=True))

parametria_comisiones.head()

In [ ]:
parametria_comisiones.shape

In [ ]:
parametria_comisiones['fuente'].unique()

In [ ]:
parametria_comisiones['meses_msi'].unique()

# COMISIONES EPA COBRADAS

In [ ]:
comisiones_epa = parametria_comisiones[(parametria_comisiones['fuente'] == 'EPA') &
                                       ((parametria_comisiones['status'].str.contains('TERMINADA')) & (parametria_comisiones['comentario'].str.contains('Sobre tasa'))) |
                                       (parametria_comisiones['comentario'].str.contains('Agregador General'))]
comisiones_epa

In [ ]:
comisiones_epa.shape

# DB

In [ ]:
# db_1 = pd.read_csv(r'G:\My Drive\RESPALDO-PC\AMEX\HISTORICO QUERIES\2021\AMEX SEPTIEMBRE 2021.csv'
#                    ,dtype={'status_code':object,'msi':object,'clave_autorizacion':object,'bin':object
#                            ,'last4':object,'order_id':object,'affiliation':object,'sub_affiliation':object
#                            ,'mcc':object})
# db_2 = pd.read_csv(r'G:\My Drive\RESPALDO-PC\AMEX\HISTORICO QUERIES\2021\AMEX AGOSTO 2021.csv'
#                    ,dtype={'status_code':object,'msi':object,'clave_autorizacion':object,'bin':object
#                            ,'last4':object,'order_id':object,'affiliation':object,'sub_affiliation':object
#                            ,'mcc':object})
db_1 = pd.read_csv(r'G:\My Drive\RESPALDO-PC\AMEX\HISTORICO QUERIES\2021\QUERIES CONSOLIDADO\AMEX AGOSTO-SEPTIEMBRE 2021.csv'
                   ,dtype={'STATUS_CODE':object,'MSI':object,'CLAVE_AUTORIZACION':object,'BIN':object
                           ,'LAST4':object,'ORDER_ID':object,'AFFILIATION':object,'SUB_AFFILIATION':object
                           ,'MCC':object})
db_2 = pd.read_csv(r'G:\My Drive\RESPALDO-PC\AMEX\HISTORICO QUERIES\2021\QUERIES CONSOLIDADO\AMEX OCTUBRE 2021.csv'
                   ,dtype={'STATUS_CODE':object,'MSI':object,'CLAVE_AUTORIZACION':object,'BIN':object
                           ,'LAST4':object,'ORDER_ID':object,'AFFILIATION':object,'SUB_AFFILIATION':object
                           ,'MCC':object})
# db_3 = pd.read_csv(r'G:\My Drive\RESPALDO-PC\AMEX\HISTORICO QUERIES\2021\QUERIES CONSOLIDADO\AMEX JUNIO-JULIO 2021.csv'
#                    ,dtype={'STATUS_CODE':object,'MSI':object,'CLAVE_AUTORIZACION':object,'BIN':object
#                            ,'LAST4':object,'ORDER_ID':object,'AFFILIATION':object,'SUB_AFFILIATION':object
#                            ,'MCC':object})
# db_4 = pd.read_csv(r'G:\My Drive\RESPALDO-PC\AMEX\HISTORICO QUERIES\2021\QUERIES CONSOLIDADO\AMEX AGOSTO-SEPTIEMBRE 2021.csv'
#                    ,dtype={'STATUS_CODE':object,'MSI':object,'CLAVE_AUTORIZACION':object,'BIN':object
#                            ,'LAST4':object,'ORDER_ID':object,'AFFILIATION':object,'SUB_AFFILIATION':object
#                            ,'MCC':object})

In [ ]:
db = pd.concat([db_1,db_2])
db.columns = db.columns.str.strip().str.lower() 
db['order_id'] = db['order_id'].str.strip()
db['order_id'] = db['order_id'].str.upper()
db['date_transaction'] = pd.to_datetime(db['date_transaction'])
db['INDEX_TOTAL'] = (db['bin'].astype(str).replace(' ', '',regex=True)+'_'
                       +db['last4'].astype(str).replace(' ', '',regex=True)+'_'
                       +db['order_id'].astype(str).replace(' ', '',regex=True)+'_'
                       +db['amount'].astype(str).replace(' ', '',regex=True))
db['INDEX_PARCIAL'] = (db['bin'].astype(str).replace(' ', '',regex=True)+'_'
                       +db['last4'].astype(str).replace(' ', '',regex=True)+'_'
                       +db['clave_autorizacion'].astype(str).replace(' ', '',regex=True)+'_'
                       +db['amount'].astype(str).replace(' ', '',regex=True))
db['index_comisiones'] = (db['affiliation_type'].astype(str).replace(' ', '',regex=True)+'_'
                       +db['msi'].str[:-6].astype(str).replace(' ', '',regex=True))
db['order_id'].fillna('SIN ORDER_ID EN DB',inplace=True)
db.head()

In [ ]:
db.shape

In [ ]:
db['msi'].unique()

In [ ]:
db['index_comisiones'].unique()

In [ ]:
db.dtypes

In [ ]:
duplicates = db[db.duplicated(['transaction_id'] , keep=False)]
duplicates.head()

In [ ]:
duplicates.shape

# DB CON COMISIONES

In [ ]:
db_final = db.merge(comisiones_epa[['index_comisiones','comision_porcentaje_mdr',
                                   'comision_porcentaje_msi','comision_porcentaje_final']],on='index_comisiones',how='left')
db_final['comision_porcentaje_final'].fillna(0,inplace=True)
db_final['comision_mdr_db'] = db_final['amount'] * db_final['comision_porcentaje_mdr']
db_final['iva_comision_mdr_db'] = db_final['comision_mdr_db'] * 0.1600
db_final['comision_msi_db'] = db_final['amount'] * db_final['comision_porcentaje_msi']
db_final['iva_comision_msi_db'] = db_final['comision_msi_db'] * 0.1600
db_final.head()

In [ ]:
db_final.shape

# EPA

In [ ]:
allfiles = glob.glob(r'G:\My Drive\EPA\EPA files\2021 EPA Amex\**\*.txt', recursive=True)
allfiles

In [ ]:
names_header=["1","2","3","4","5","6","7","8","9","10","A","12","13","14","15","16","17","18","19","order_id",
              "21","22","23","24","25","26","27","28"]

In [ ]:
epa = pd.concat([pd.read_csv(f,names=names_header, sep=',',dtype=object) for f in allfiles])
epa['order_id'] = epa['order_id'].str.strip()
epa.head()

In [ ]:
epa.shape

In [ ]:
epa['6'].unique()

In [ ]:
epa_0 = epa[(epa['6'] == '0')]
epa_1 = epa[(epa['6'] == '1')]
epa_3 = epa[(epa['6'] == '3')]
epa_4 = epa[(epa['6'] == '4')]
epa_5 = epa[(epa['6'] == '5')]
epa_9 = epa[(epa['6'] == '9')]

## EPA 0

In [ ]:
epa_0.head()

In [ ]:
epa_0.shape

## EPA 1

In [ ]:
epa_1['2'] = pd.to_datetime(epa_1['2'], format='%Y%m%d').dt.date
epa_1['2'] = pd.to_datetime(epa_1['2'])
epa_1.sort_values(by=['2'],inplace=True)
epa_1.head()

In [ ]:
epa_1.shape

In [ ]:
epa_1_final = epa_1[['6','2','8','15','16','17','18','19']]
epa_1_final.head()

In [ ]:
epa_1_final['8'] = epa_1_final['8'].astype(float)
epa_1_final['15'] = epa_1_final['15'].astype(float)
epa_1_final['16'] = epa_1_final['16'].astype(float)
epa_1_final['17'] = epa_1_final['17'].astype(float)
epa_1_final['18'] = epa_1_final['18'].astype(float)
epa_1_final['19'] = epa_1_final['19'].astype(float)

In [ ]:
epa_1_final.rename(columns ={'6':'Type','2':'Payment_date'},inplace=True)

In [ ]:
epa_1_final['Payment_amount'] = epa_1_final['8']/100
epa_1_final['Gross_amount'] = epa_1_final['15']/100
epa_1_final['Discount_amount'] = epa_1_final['16']/100
epa_1_final['Taxes_amount'] = epa_1_final['17']/100
epa_1_final['Discount_aceleration_amount'] = epa_1_final['18']/100
epa_1_final['Net_amount'] = epa_1_final['19']/100
epa_1_final.head()

In [ ]:
epa_1_excel = epa_1_final[(epa_1_final['Payment_date'] >= start_date) & (epa_1_final['Payment_date'] <= final_date)]

In [ ]:
epa_1_excel_final = epa_1_excel.groupby(['Payment_date']).agg({'Payment_amount':sum,'Gross_amount':sum,'Discount_amount':sum
                                                        ,'Taxes_amount':sum,'Discount_aceleration_amount':sum
                                                        ,'Net_amount':sum}).reset_index()
epa_1_excel_final.head()

In [ ]:
epa_1_excel_final.shape

In [ ]:
epa_1_excel_final.to_excel('EPA_1.xlsx')

## EPA 3

In [ ]:
epa_3['2'] = pd.to_datetime(epa_3['2'], format='%Y%m%d').dt.date
epa_3['2'] = pd.to_datetime(epa_3['2'])
epa_3.sort_values(by=['2'],inplace=True)
epa_3.head()

In [ ]:
epa_3.shape

In [ ]:
epa_3_final = epa_3[['6','2','10','A','12','13','14','15','16','25','26','27','28']]
epa_3_final.reset_index(drop=True, inplace=True)
epa_3_final.head()

In [ ]:
epa_3_final['10'] = epa_3_final['10'].astype(float)
epa_3_final['A'] = epa_3_final['A'].astype(float)
epa_3_final['12'] = epa_3_final['12'].astype(float)
epa_3_final['13'] = epa_3_final['13'].astype(float)
epa_3_final['14'] = epa_3_final['14'].astype(float)
epa_3_final['15'] = epa_3_final['15'].astype(float)
epa_3_final['16'] = epa_3_final['16'].astype(float)
epa_3_final['25'] = epa_3_final['25'].astype(float)
epa_3_final['26'] = epa_3_final['26'].astype(float)
epa_3_final['27'] = epa_3_final['27'].astype(float)
epa_3_final['28'] = epa_3_final['28'].astype(float)

In [ ]:
epa_3_final.rename(columns ={'6':'Type','2':'Payment_date','16':'Register_number'},inplace=True)

In [ ]:
epa_3_final['Gross_amount'] = epa_3_final['10']/100
epa_3_final['Gross_amount_amex'] = epa_3_final['A']/100
epa_3_final['Discount_amount'] = epa_3_final['12']/100
epa_3_final['Taxes_amount'] = epa_3_final['13']/100
epa_3_final['Discount_aceleration_amount'] = epa_3_final['14']/100
epa_3_final['Net_amount'] = epa_3_final['15']/100
epa_3_final['Aceleration_net_amount'] = epa_3_final['25']/100
epa_3_final['Gross_amount_trans_cambio'] = epa_3_final['26']/100
epa_3_final['Credit_gross_amount'] = epa_3_final['27']/100
epa_3_final['Unknown'] = epa_3_final['28']/100
epa_3_final.head()

In [ ]:
epa_3_excel = epa_3_final[(epa_3_final['Payment_date'] >= start_date) & (epa_3_final['Payment_date'] <= final_date)]

In [ ]:
epa_3_excel_final = epa_3_excel.groupby(['Payment_date']).agg({'Register_number':sum,'Gross_amount':sum,'Gross_amount_amex':sum
                                                               ,'Discount_amount':sum,'Taxes_amount':sum
                                                               ,'Discount_aceleration_amount':sum,'Net_amount':sum
                                                               ,'Aceleration_net_amount':sum
                                                               ,'Gross_amount_trans_cambio':sum,'Credit_gross_amount':sum
                                                               ,'Unknown':sum}).reset_index()
epa_3_excel_final.head()

In [ ]:
epa_3_excel_final.shape

In [ ]:
epa_3_excel_final.to_csv('EPA_3.csv')

## EPA 4

In [ ]:
epa_4['8'] = pd.to_datetime(epa_4['8'], format='%Y%m%d').dt.date
epa_4['2'] = pd.to_datetime(epa_4['2'], format='%Y%m%d').dt.date

In [ ]:
epa_4['8'] = pd.to_datetime(epa_4['8'])
epa_4['2'] = pd.to_datetime(epa_4['2'])

In [ ]:
epa_4.head()

In [ ]:
epa_4.shape

In [ ]:
epa_4.to_csv('EPA_4.csv',index=False)

In [ ]:
txn_epa = epa_4[['order_id','6','8','A','12','22','23','2','15']]
txn_epa["12"] = txn_epa['12'].astype(float)
txn_epa["22"] = txn_epa['22'].astype(float)
txn_epa["23"] = txn_epa['23'].astype(float)


In [ ]:
txn_epa['total_amount_bruto'] = txn_epa['12']/100
txn_epa['fee_amount_mdr'] = -(txn_epa['total_amount_bruto']*0.0240)
txn_epa['fee_amount_msi'] = txn_epa['22']/100
txn_epa['iva_msi'] = txn_epa['fee_amount_msi']*0.1600
txn_epa['iva_mdr'] = txn_epa['fee_amount_mdr']*0.1600
txn_epa['bin_bank'] = txn_epa['A'].str[:6].astype(str)
txn_epa['last4_bank'] = txn_epa['A'].str[-8:-4].astype(str)

In [ ]:
txn_epa.rename(columns ={'6':'Type','8':'Date_process_roc','12':'Total_amount_charge_roc',
                         '22':'Svc_fee_amount_roc','23':'Svc_fee_amount_accel_roc','2':'roc_deposit_date'},inplace=True)
txn_epa['INDEX_TOTAL'] = (txn_epa['bin_bank'].astype(str).replace(' ', '',regex=True)+'_'
                            +txn_epa['last4_bank'].astype(str).replace(' ', '',regex=True)+'_'
                            +txn_epa['order_id'].astype(str).replace(' ', '',regex=True)+'_'
                            +txn_epa['total_amount_bruto'].astype(str).replace(' ', '',regex=True))
# txn_epa['INDEX_PARCIAL'] = (txn_epa['bin_bank'].astype(str).replace(' ', '',regex=True)+'_'
#                             +txn_epa['last4_bank'].astype(str).replace(' ', '',regex=True)+'_'
#                             +txn_epa['codigo_aprobacion'].astype(str).replace(' ', '',regex=True)+'_'
#                             +txn_epa['total_amount_bruto'].astype(str).replace(' ', '',regex=True))
txn_epa.fillna('SIN ORDER_ID EN EPA',inplace=True)
txn_epa.head()

In [ ]:
txn_epa.shape

In [ ]:
# txn_epa.to_excel('EPA CONSOLIDADO.xlsx',index=False)

### DATA PARA PROVISION

In [ ]:
start_date_prov = '2021-10-25'

In [ ]:
txn_epa_prov = txn_epa[(txn_epa['Date_process_roc'] >= start_date_prov)]
txn_epa_prov.head()

In [ ]:
txn_epa_prov.shape

In [ ]:
txn_epa_prov.to_csv('AMEX PROVISION EPA OCTUBRE 2021.csv',index=False)

## EPA 5

In [ ]:
epa_5['2'] = pd.to_datetime(epa_5['2'], format='%Y%m%d').dt.date

In [ ]:
epa_5['2'] = pd.to_datetime(epa_5['2'])

In [ ]:
epa_5.head()

In [ ]:
epa_5.shape

In [ ]:
txn_epa_adj = epa_5[['2','8','9','10','A','12','13','14','15','16','18']]
txn_epa_adj.rename(columns ={'2':'roc_deposit_date','8':'clave_autorizacion','14':'no_tarjeta','15':'adj_id','16':'motivo_adj'},inplace=True) 
txn_epa_adj["9"] = txn_epa_adj['9'].astype(float)
txn_epa_adj["10"] = txn_epa_adj['10'].astype(float)
txn_epa_adj["A"] = txn_epa_adj['A'].astype(float)
txn_epa_adj["12"] = txn_epa_adj['12'].astype(float)
txn_epa_adj["13"] = txn_epa_adj['13'].astype(float)


In [ ]:
txn_epa_adj['importe_adj_bruto'] = txn_epa_adj['9']/100
txn_epa_adj['tasa_adj'] = txn_epa_adj['10']/100
txn_epa_adj['importe_impuesto'] = txn_epa_adj['A']/100
txn_epa_adj['tasa_servicio_plan_n'] = txn_epa_adj['12']/100
txn_epa_adj['importe_adj_neto'] = txn_epa_adj['13']/100
txn_epa_adj['bin_bank'] = txn_epa_adj['no_tarjeta'].str[:6].astype(str)
txn_epa_adj['last4_bank'] = txn_epa_adj['no_tarjeta'].str[-8:-4].astype(str)

In [ ]:
txn_epa_adj.head()

In [ ]:
txn_epa_adj.shape

In [ ]:
txn_epa_adj.to_excel(r'ADJ CONSOLIDADO.xlsx',index=False)

In [ ]:
txn_epa_adj_final = txn_epa_adj[(txn_epa_adj['roc_deposit_date'] >= start_date) & (txn_epa_adj['roc_deposit_date'] <= final_date)]

## EPA 9

In [ ]:
epa_9.head()

In [ ]:
epa_9.shape

# CONCILIACION ALL

In [ ]:
cruce_1 = txn_epa.merge(db_final[['order_id','transaction_id','date_transaction','amount','status_code','msi','bin','last4',
                                  'clave_autorizacion','comision_porcentaje_mdr','comision_porcentaje_msi',
                                  'comision_porcentaje_final','comision_mdr_db','iva_comision_mdr_db',
                                  'comision_msi_db','iva_comision_msi_db']],on='order_id',how='left')
cruce_1[['transaction_id','status_code','msi']].fillna('NO ENCONTRADO',inplace=True)
cruce_1['amount'].fillna(0,inplace=True)
cruce_1['status_code'].fillna('NO ENCONTRADO',inplace=True)
cruce_1.loc[~cruce_1['status_code'].str.contains('3'),'amount_new'] = cruce_1['amount']
cruce_1.loc[cruce_1['status_code'].str.contains('3'),'amount_new'] = cruce_1['amount']*-1
cruce_1['comparison'] = cruce_1['total_amount_bruto'] == cruce_1['amount_new']
cruce_1.head()

In [ ]:
cruce_1.shape

In [ ]:
cruce_1['status_code'].unique()

# PAYMENTS CRUCE AMEX VS CLIP

In [ ]:
txn_epa_deposit = cruce_1[(cruce_1['roc_deposit_date'] >= start_date) & (cruce_1['roc_deposit_date'] <= final_date)]
txn_epa_deposit.head()

In [ ]:
txn_epa_deposit.shape

In [ ]:
txn_epa_deposit['roc_deposit_date'].unique()

In [ ]:
duplicates_1 = txn_epa_deposit[txn_epa_deposit.duplicated(['order_id'] , keep=False)]
duplicates_1.sort_values(by=['order_id','Date_process_roc'],inplace=True)
duplicates_1.head()


In [ ]:
duplicates_1.shape

In [ ]:
txn_epa_deposit.to_csv('EJEMPLO COMISIONES.csv')

### COMPARISON

In [ ]:
comparison_1 = txn_epa_deposit[(txn_epa_deposit['comparison'] == False)]
comparison_1.head()

In [ ]:
comparison_1.shape

### ADJUSTMENTS

In [ ]:
txn_epa_adj_trunc = txn_epa_adj[(txn_epa_adj['roc_deposit_date'] >= start_date) &
                                (txn_epa_adj['roc_deposit_date'] <= final_date)]

In [ ]:
adj_montos = txn_epa_adj_trunc[(txn_epa_adj_trunc['motivo_adj'] != 'MONTO DE SERVICIO')]
adj_montos.head()

In [ ]:
adj_montos.shape

In [ ]:
deposit_date_adj = adj_montos.groupby(['roc_deposit_date']).agg({'importe_adj_bruto':sum,'tasa_adj':sum
                                                                 ,'importe_impuesto':sum,'tasa_servicio_plan_n':sum
                                                                 ,'importe_adj_neto':sum}).reset_index()
deposit_date_adj.head()

In [ ]:
deposit_date_adj.shape

In [ ]:
adj_monto_servicio = txn_epa_adj_trunc[(txn_epa_adj_trunc['motivo_adj'] == 'MONTO DE SERVICIO')]
adj_monto_servicio.head()

In [ ]:
adj_monto_servicio.shape

In [ ]:
pivot_monto_servicio = adj_monto_servicio.groupby(['roc_deposit_date']).agg({'importe_adj_bruto':sum,'tasa_adj':sum
                                                                 ,'importe_impuesto':sum,'tasa_servicio_plan_n':sum
                                                                 ,'importe_adj_neto':sum}).reset_index()
pivot_monto_servicio.head()

In [ ]:
pivot_monto_servicio.shape

#### ADJUSTMENTS POSITIVOS

In [ ]:
adj_montos_positivos = adj_montos[(adj_montos['importe_adj_bruto'] >= 0)]

In [ ]:
deposit_date_adj_positivos = adj_montos_positivos.groupby(['roc_deposit_date']).agg({'importe_adj_bruto':sum,'tasa_adj':sum
                                                                 ,'importe_impuesto':sum,'tasa_servicio_plan_n':sum
                                                                 ,'importe_adj_neto':sum}).reset_index()
deposit_date_adj_positivos.head()

In [ ]:
deposit_date_adj_positivos.shape

In [ ]:
adj_monto_servicio_positivos = adj_monto_servicio[(adj_monto_servicio['importe_adj_neto'] >= 0)]
adj_monto_servicio_positivos.head()

In [ ]:
adj_monto_servicio_positivos.shape

In [ ]:
pivot_monto_servicio_positivos = adj_monto_servicio_positivos.groupby(['roc_deposit_date']).agg({'importe_adj_bruto':sum,
                                                                                                 'tasa_adj':sum,
                                                                                                 'importe_impuesto':sum,
                                                                                                 'tasa_servicio_plan_n':sum,
                                                                                                 'importe_adj_neto':sum}).reset_index()
pivot_monto_servicio_positivos.head()

In [ ]:
pivot_monto_servicio_positivos.shape

#### ADJUSTMENTS NEGATIVOS

In [ ]:
adj_montos_negativos = adj_montos[(adj_montos['importe_adj_bruto'] < 0)]

In [ ]:
deposit_date_adj_negativos = adj_montos_negativos.groupby(['roc_deposit_date']).agg({'importe_adj_bruto':sum,
                                                                                     'tasa_adj':sum,
                                                                                     'importe_impuesto':sum,
                                                                                     'tasa_servicio_plan_n':sum,
                                                                                     'importe_adj_neto':sum}).reset_index()
deposit_date_adj_negativos.head()

In [ ]:
deposit_date_adj_negativos.shape

In [ ]:
adj_monto_servicio_negativos = adj_monto_servicio[(adj_monto_servicio['importe_adj_neto'] < 0)]
adj_monto_servicio_negativos.head()

In [ ]:
adj_monto_servicio_negativos.shape

In [ ]:
pivot_monto_servicio_negativos = adj_monto_servicio_negativos.groupby(['roc_deposit_date']).agg({'importe_adj_bruto':sum,
                                                                                                 'tasa_adj':sum,
                                                                                                 'importe_impuesto':sum,
                                                                                                 'tasa_servicio_plan_n':sum,
                                                                                                 'importe_adj_neto':sum}).reset_index()
pivot_monto_servicio_negativos.head()

In [ ]:
pivot_monto_servicio_negativos.shape

### TRANSACCIONES POSITIVAS

In [ ]:
deposit_success_txn = txn_epa_deposit[(txn_epa_deposit['total_amount_bruto'] >= 0)]

In [ ]:
deposit_success_cruce_1 = deposit_success_txn.groupby(['roc_deposit_date']).agg({'total_amount_bruto':sum,
                                                                                 'fee_amount_mdr':sum,
                                                                                 'fee_amount_msi':sum,
                                                                                 'iva_mdr':sum,
                                                                                 'iva_msi':sum,'comision_mdr_db':sum,
                                                                                 'iva_comision_mdr_db':sum,
                                                                                 'comision_msi_db':sum,
                                                                                 'iva_comision_msi_db':sum}).reset_index()
deposit_success_cruce_1.head()

In [ ]:
deposit_success_cruce_1.shape

In [ ]:
deposit_success_cruce_2 = deposit_success_cruce_1.merge(deposit_date_adj_positivos[['roc_deposit_date','importe_adj_bruto',
                                                                                    'tasa_adj','importe_impuesto',
                                                                                    'tasa_servicio_plan_n','importe_adj_neto']]
                                                  ,on='roc_deposit_date',how='left')
deposit_success_cruce_2['importe_adj_bruto'].fillna(0,inplace=True)
deposit_success_cruce_2['tasa_adj'].fillna(0,inplace=True)
deposit_success_cruce_2['importe_impuesto'].fillna(0,inplace=True)
deposit_success_cruce_2['tasa_servicio_plan_n'].fillna(0,inplace=True)
deposit_success_cruce_2['importe_adj_neto'].fillna(0,inplace=True)
deposit_success_cruce_2['deposit_amount'] = (deposit_success_cruce_2['total_amount_bruto']
                                             + deposit_success_cruce_2['fee_amount_mdr']
                                             + deposit_success_cruce_2['fee_amount_msi']
                                             + deposit_success_cruce_2['iva_mdr']
                                             + deposit_success_cruce_2['iva_msi']
                                             + deposit_success_cruce_2['importe_adj_bruto']
                                             + deposit_success_cruce_2['tasa_adj']
                                             + deposit_success_cruce_2['importe_impuesto']
                                             + deposit_success_cruce_2['tasa_servicio_plan_n'])
    
deposit_success_cruce_2.head()

In [ ]:
deposit_success_cruce_2.shape

In [ ]:
deposit_success_cruce_final = deposit_success_cruce_2[['roc_deposit_date','total_amount_bruto','fee_amount_mdr',
                                                       'fee_amount_msi','iva_mdr','iva_msi','comision_mdr_db',
                                                       'iva_comision_mdr_db','comision_msi_db','iva_comision_msi_db',
                                                       'importe_adj_bruto','tasa_adj','importe_impuesto','tasa_servicio_plan_n',
                                                       'deposit_amount']]
deposit_success_cruce_final.head()

In [ ]:
deposit_success_cruce_final.shape

In [ ]:
deposit_success_cruce_final.to_excel('DEPOSITOS POSITIVOS.xlsx')

### CANCELACIONES

In [ ]:
deposit_refunded_txn = txn_epa_deposit[(txn_epa_deposit['total_amount_bruto'] < 0)]

In [ ]:
deposit_refunded_cruce_1 = deposit_refunded_txn.groupby(['roc_deposit_date']).agg({'total_amount_bruto':sum,
                                                                                   'fee_amount_mdr':sum,
                                                                                   'fee_amount_msi':sum,
                                                                                   'iva_mdr':sum,
                                                                                   'iva_msi':sum,'comision_mdr_db':sum,
                                                                                   'iva_comision_mdr_db':sum,
                                                                                   'comision_msi_db':sum,
                                                                                   'iva_comision_msi_db':sum}).reset_index()
deposit_refunded_cruce_1.head()

In [ ]:
deposit_refunded_cruce_1.shape

In [ ]:
deposit_refunded_cruce_2 = deposit_refunded_cruce_1.merge(deposit_date_adj_negativos[['roc_deposit_date','importe_adj_bruto',
                                                                                      'tasa_adj','importe_impuesto',
                                                                                      'tasa_servicio_plan_n',
                                                                                      'importe_adj_neto']],
                                                          on='roc_deposit_date',how='right')
deposit_refunded_cruce_2.sort_values(by=['roc_deposit_date'],inplace=True)
deposit_refunded_cruce_2['total_amount_bruto'].fillna(0,inplace=True)
deposit_refunded_cruce_2['fee_amount_mdr'].fillna(0,inplace=True)
deposit_refunded_cruce_2['fee_amount_msi'].fillna(0,inplace=True)
deposit_refunded_cruce_2['iva_mdr'].fillna(0,inplace=True)
deposit_refunded_cruce_2['iva_msi'].fillna(0,inplace=True)
deposit_refunded_cruce_2['importe_adj_bruto'].fillna(0,inplace=True)
deposit_refunded_cruce_2['tasa_adj'].fillna(0,inplace=True)
deposit_refunded_cruce_2['importe_impuesto'].fillna(0,inplace=True)
deposit_refunded_cruce_2['tasa_servicio_plan_n'].fillna(0,inplace=True)
deposit_refunded_cruce_2['importe_adj_neto'].fillna(0,inplace=True)
deposit_refunded_cruce_2['comision_mdr_db'].fillna(0,inplace=True)
deposit_refunded_cruce_2['iva_comision_mdr_db'].fillna(0,inplace=True)
deposit_refunded_cruce_2['comision_msi_db'].fillna(0,inplace=True)
deposit_refunded_cruce_2['iva_comision_mdr_db'].fillna(0,inplace=True)
deposit_refunded_cruce_2['deposit_amount'] = (deposit_refunded_cruce_2['total_amount_bruto']
                                             + deposit_refunded_cruce_2['fee_amount_mdr']
                                             + deposit_refunded_cruce_2['fee_amount_msi']
                                             + deposit_refunded_cruce_2['iva_mdr']
                                             + deposit_refunded_cruce_2['iva_msi']
                                             + deposit_refunded_cruce_2['importe_adj_bruto']
                                             + deposit_refunded_cruce_2['tasa_adj']
                                             + deposit_refunded_cruce_2['importe_impuesto']
                                             + deposit_refunded_cruce_2['tasa_servicio_plan_n'])
    
deposit_refunded_cruce_2.head()

In [ ]:
deposit_refunded_cruce_2.shape

In [ ]:
deposit_refunded_cruce_final = deposit_refunded_cruce_2[['roc_deposit_date','total_amount_bruto','fee_amount_mdr',
                                                         'fee_amount_msi','iva_mdr','iva_msi','comision_mdr_db',
                                                         'iva_comision_mdr_db','comision_msi_db','iva_comision_msi_db',
                                                         'importe_adj_bruto','tasa_adj','importe_impuesto',
                                                         'tasa_servicio_plan_n','deposit_amount']]
deposit_refunded_cruce_final.head()

In [ ]:
deposit_refunded_cruce_final.shape

In [ ]:
deposit_refunded_cruce_final.to_excel('DEPOSITOS NEGATIVOS.xlsx')

### DEPOSITOS FINALES

In [ ]:
deposit_date_cruce_1 = txn_epa_deposit.groupby(['roc_deposit_date']).agg({'total_amount_bruto':sum,
                                                                          'fee_amount_mdr':sum,
                                                                          'fee_amount_msi':sum,
                                                                          'iva_mdr':sum,
                                                                          'iva_msi':sum,
                                                                          'comision_mdr_db':sum,
                                                                          'iva_comision_mdr_db':sum,
                                                                          'comision_msi_db':sum,
                                                                          'iva_comision_msi_db':sum}).reset_index()
deposit_date_cruce_1.head()

In [ ]:
deposit_date_cruce_1.shape

In [ ]:
deposit_date_cruce_2 = deposit_date_cruce_1.merge(deposit_date_adj[['roc_deposit_date','importe_adj_bruto','tasa_adj',
                                                                    'importe_impuesto','tasa_servicio_plan_n',
                                                                    'importe_adj_neto']],
                                                  on='roc_deposit_date',how='left')
deposit_date_cruce_2['importe_adj_bruto'].fillna(0,inplace=True)
deposit_date_cruce_2['tasa_adj'].fillna(0,inplace=True)
deposit_date_cruce_2['importe_impuesto'].fillna(0,inplace=True)
deposit_date_cruce_2['tasa_servicio_plan_n'].fillna(0,inplace=True)
deposit_date_cruce_2['importe_adj_neto'].fillna(0,inplace=True)
deposit_date_cruce_2['deposit_amount'] = (deposit_date_cruce_2['total_amount_bruto'] + deposit_date_cruce_2['fee_amount_mdr']
                                          + deposit_date_cruce_2['fee_amount_msi'] + deposit_date_cruce_2['iva_mdr']
                                          + deposit_date_cruce_2['iva_msi'] + deposit_date_cruce_2['importe_adj_bruto']
                                          + deposit_date_cruce_2['tasa_adj'] + deposit_date_cruce_2['importe_impuesto']
                                          + deposit_date_cruce_2['tasa_servicio_plan_n'])
    
deposit_date_cruce_2.head()

In [ ]:
deposit_date_cruce_final = deposit_date_cruce_2[['roc_deposit_date','total_amount_bruto','fee_amount_mdr','fee_amount_msi',
                                                 'iva_mdr','iva_msi','comision_mdr_db','iva_comision_mdr_db','comision_msi_db',
                                                 'iva_comision_msi_db','importe_adj_bruto','tasa_adj','importe_impuesto',
                                                 'tasa_servicio_plan_n','deposit_amount']]
deposit_date_cruce_final.head()

In [ ]:
deposit_date_cruce_final.shape

# TRANSACTIONS AMEX - CLIP

In [ ]:
txn_epa_txn = cruce_1[(cruce_1['Date_process_roc'] >= start_date) & (cruce_1['Date_process_roc'] <= final_date)]
txn_epa_txn.sort_values(by=['Date_process_roc','order_id'],inplace=True)
txn_epa_txn.head()

In [ ]:
txn_epa_txn.shape

In [ ]:
duplicates_2 = txn_epa_txn[txn_epa_txn.duplicated(['order_id'] , keep=False)]
duplicates_2.sort_values(by=['order_id','Date_process_roc'],inplace=True)
duplicates_2.head()


In [ ]:
duplicates_2.shape

In [ ]:
txn_epa_txn.drop_duplicates(subset ='order_id', keep='last', inplace=True)
txn_epa_txn.shape

In [ ]:
comparison_2 = txn_epa_txn[(txn_epa_txn['comparison'] == False)]
comparison_2.head()

In [ ]:
comparison_2.shape

# GROUP BY TRANSACTION DATE

In [ ]:
txn_txn_date_cruce_1 = txn_epa_txn.groupby(['Date_process_roc']).agg({'total_amount_bruto':sum,
                                                                      'fee_amount_mdr':sum,
                                                                      'fee_amount_msi':sum,
                                                                      'iva_mdr':sum,
                                                                      'iva_msi':sum,
                                                                      'comision_mdr_db':sum,
                                                                      'iva_comision_mdr_db':sum,
                                                                      'comision_msi_db':sum,
                                                                      'iva_comision_msi_db':sum,
                                                                      'amount_new':sum,
                                                                      'order_id':'count'}).reset_index()
txn_txn_date_cruce_1.rename(columns ={'order_id':'number_transactions'}, inplace=True)
txn_txn_date_cruce_1['diff'] = txn_txn_date_cruce_1['total_amount_bruto'] - txn_txn_date_cruce_1['amount_new']
txn_txn_date_cruce_1.head()


In [ ]:
txn_txn_date_cruce_1.shape

# GROUP BY TRANSACTION DATE AND STATUS CODE

In [ ]:
txn_txn_status_cruce_1 = txn_epa_txn.groupby(['Date_process_roc','status_code']).agg({'total_amount_bruto':sum,
                                                                                      'fee_amount_mdr':sum,
                                                                                      'fee_amount_msi':sum,
                                                                                      'iva_mdr':sum,
                                                                                      'iva_msi':sum,
                                                                                      'comision_mdr_db':sum,
                                                                                      'iva_comision_mdr_db':sum,
                                                                                      'comision_msi_db':sum,
                                                                                      'iva_comision_msi_db':sum,
                                                                                      'amount_new':sum,
                                                                                      'order_id':'count'}).reset_index()
txn_txn_status_cruce_1.rename(columns ={'order_id':'number_transactions'}, inplace=True)
txn_txn_status_cruce_1['diff'] = txn_txn_status_cruce_1['total_amount_bruto'] - txn_txn_status_cruce_1['amount_new']
txn_txn_status_cruce_1.head()


In [ ]:
txn_txn_status_cruce_1.dtypes

In [ ]:
txn_txn_status_cruce_1.shape

### STATUS 1

In [ ]:
txn_txn_status_cruce_1_status_1 = txn_txn_status_cruce_1[(txn_txn_status_cruce_1['status_code'] == '1')]
txn_txn_status_cruce_1_status_1.head()

In [ ]:
txn_txn_status_cruce_1_status_1.shape

### STATUS 3

In [ ]:
txn_txn_status_cruce_1_status_3 = txn_txn_status_cruce_1[(txn_txn_status_cruce_1['status_code'] == '3')]
txn_txn_status_cruce_1_status_3.head()

In [ ]:
txn_txn_status_cruce_1_status_3.shape

# POSIBLES DISPUTAS

In [ ]:
posibles_disputas = txn_epa_txn[(txn_epa_txn['status_code'] != '1') & (txn_epa_txn['status_code'] != '3')]
posibles_disputas.head()


In [ ]:
posibles_disputas.shape

### POSIBLES DISPUTAS BY DATE

In [ ]:
txn_txn_status_cruce_1_posibles_disputas = posibles_disputas.groupby(['Date_process_roc']).agg({'total_amount_bruto':sum,
                                                                                                'fee_amount_mdr':sum,
                                                                                                'fee_amount_msi':sum,
                                                                                                'iva_mdr':sum,'iva_msi':sum,
                                                                                                'amount_new':sum,
                                                                                                'comision_mdr_db':sum,
                                                                                                'iva_comision_mdr_db':sum,
                                                                                                'comision_msi_db':sum,
                                                                                                'iva_comision_msi_db':sum,
                                                                                                'order_id':'count'}).reset_index()
txn_txn_status_cruce_1_posibles_disputas.head()

In [ ]:
txn_txn_status_cruce_1_posibles_disputas.shape

# EXCEL

## PAYOUTS

In [ ]:
writer = pd.ExcelWriter('EPA PAYOUTS OCTUBRE CON COMISIONES 2021.xlsx', engine='xlsxwriter', options={'encoding':'UTF-8'})
deposit_date_cruce_final.to_excel(writer, sheet_name='PAYOUTS',index=False)
deposit_success_cruce_1.to_excel(writer, sheet_name='PAYOUTS SUCCESS TXN',index=False)
deposit_refunded_cruce_1.to_excel(writer, sheet_name='PAYOUTS REFUNDED TXN',index=False)
epa_1_excel_final.to_excel(writer, sheet_name='EPA PAYMENT RECORD',index=False)
epa_3_excel_final.to_excel(writer, sheet_name='EPA SUM OF CHARGE',index=False)
txn_epa_deposit.to_excel(writer, sheet_name='PAYOUTS DETAIL',index=False)
duplicates_1.to_excel(writer, sheet_name='PAYOUTS DUPLICATES',index=False)
comparison_1.to_excel(writer, sheet_name='PAYOUTS COMPARISON',index=False)
txn_epa_adj_final.to_excel(writer, sheet_name='ADJUSTMENS DETAIL',index=False)
deposit_date_adj_positivos.to_excel(writer, sheet_name='ADJUSTMENTS POSITIVOS',index=False)
pivot_monto_servicio_positivos.to_excel(writer, sheet_name='MONTO SERVICIO POSIVOS',index=False)
deposit_date_adj_negativos.to_excel(writer, sheet_name='ADJUSTMENTS NEGATIVOS',index=False)
pivot_monto_servicio_negativos.to_excel(writer, sheet_name='MONTO SERVICIO NEGATIVOS',index=False)
writer.save()
writer.close()

## TRANSACCIONES

In [ ]:
writer = pd.ExcelWriter('EPA TRANSACCIONES CON COMISIONES OCTUBRE 2021.xlsx', engine='xlsxwriter', options={'encoding':'UTF-8'})
txn_epa_txn.to_excel(writer, sheet_name='TRANSACTIONS DETAIL',index=False)
duplicates_2.to_excel(writer, sheet_name='TRANSACTIONS DUPLICATES',index=False)
comparison_2.to_excel(writer, sheet_name='TRANSACTIONS COMPARISON',index=False)
txn_txn_date_cruce_1.to_excel(writer, sheet_name='TXN DATE',index=False)
txn_txn_status_cruce_1.to_excel(writer, sheet_name='TXN DATE AND STATUS',index=False)
txn_txn_status_cruce_1_status_1.to_excel(writer, sheet_name='STATUS 1',index=False)
txn_txn_status_cruce_1_status_3.to_excel(writer, sheet_name='STATUS 3',index=False)
txn_txn_status_cruce_1_posibles_disputas.to_excel(writer, sheet_name='POSIBLES DISPUTAS BY DATE',index=False)
posibles_disputas.to_excel(writer, sheet_name='POSIBLES DISPUTAS',index=False)
writer.save()
writer.close()